# SD202 TP2 - Normalization and SQL

The objectives of this TP are the following:

1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this lab, we are going to use a database containing wine information related to 'production' and 'sales'. 

Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


We need to prepare the SQL environment:

In [1]:
import sqlite3

In [2]:
def printSchema(connection):
    ### Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [3]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
Buying:
  0: NV(NUM) *1
  1: NB(NUM) *2
  2: DATES(TEXT)
  3: LIEU(TEXT)
  4: QTE(NUM)
Clients:
  0: NB(NUM) *1
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
Lieux:
  0: LIEU(TEXT) *1
  1: REGION(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(NUM)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
Producers:
  0: NP(NUM) *1
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
Quantities:
  0: NV(NUM) *1
  1: NP(NUM) *2
  2: QTE(NUM)
Wines:
  0: NV(NUM) *1
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


We recommend inline __%sql__ as an alternative to sqlite3 package

In [4]:
%load_ext sql
%sql sqlite:///wine.db

'Connected: @wine.db'

Now, we can see the content of the tables using SQL queries:

In [5]:
%sql SELECT DISTINCT NV, CRU, MILL, DEGRE FROM MASTER1;

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
None,None,None,None
1,Mercurey,1980,11.5
2,Julienas,1974,11.3
3,Savigny les Beaunes,1978,12.1
4,Mercurey,1980,10.9
5,Pommard,1976,11.7
6,Mercurey,1981,11.2
7,Grands Echezeaux,1968,11.7
8,Cotes de Beaune Villages,1975,12.3
9,Chapelle Chambertin,1973,11.9


# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables Master1 and Master2 are in the First Normal Form (1NF) and suffer from data redundancy. 

__1.1__ Given the set of the following FDs:

NV -> CRU, DEGRE, MILL

NP -> NOM, PRENOM, REGION

NV, NP -> QTE

Convert table Master1 to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.

* Explain your answer
* Describe the schema of new tables and how they relate

This table is not in Second Normal Form (2NF), since CRU, DEGREE, MILL only depends on NV, which is a subpart of the key (NV, NP), and NOM, PRENOM, REGION only depends on NP, which is also a subpart of the key (NV, NP).

To convert the table Master1 to the Second Normal Form, we can create three tables:
- Wines, containing NV, CRU, DEGREE and MILL (NV is the primary key of the table)
- Producers, containing NP, NOM, PRENOM, and REGION (NP is the primary key of the table)
- Quantities, containing NV, NP and QTE ({NV, NP} is the primary key of the table)

If we create these three tables, all of them are also in Third Normal Form (3NF), since there aren't any functional dependencies between non-key attributes.

__1.2__ Given the set of the following FDs:

NV -> CRU, DEGRE, MILL

NB -> NOM, PRENOM, TYPE

NV, NB -> DATES, LIEU, QTE

LIEU -> REGION 

Convert table Master2 to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable. 
* Explain your answer
* Describe the schema of new tables and how they relate

This second table is also not in the Second Normal Form, for the same reasons as for Master 1. To convert the table Master2 to the Second Normal Form, we can create four tables:

- Wines, containing NV, CRU, DEGREE and MILL (NV is the primary key of the table)
- Clients, containing NB, NOM, PRENOM and TYPE (NB is the primary key of the table)
- Lieux, containing LIEU and REGION (LIEU is the primary key of the table)
- Buying, containing NV, NB, DATES, LIEU and QTE ({NV, NB} is the primary key of the table, and LIEU is a foreign key to the table Lieux)

These tables are also in Third Normal Form (3NF), since there aren't any functional dependencies between non-key attributes. Since the tables "Wines" from Master1 and Master2 contains the same types of data, we can have only one table "Wines", in which we will insert the data from Master1 and Master2.

Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables. A table can be created from the result of a query. In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [6]:
%sql DROP TABLE IF EXISTS dummy;

# Create dummy table
%sql CREATE TABLE dummy AS \
SELECT DISTINCT DEGRE \
FROM MASTER1;

print("\nContent of the database")
printSchema(conn)

print("\nContent of dummy")
%sql SELECT * FROM dummy

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.

Content of the database
Buying:
  0: NV(NUM) *1
  1: NB(NUM) *2
  2: DATES(TEXT)
  3: LIEU(TEXT)
  4: QTE(NUM)
Clients:
  0: NB(NUM) *1
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
Lieux:
  0: LIEU(TEXT) *1
  1: REGION(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(NUM)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
Producers:
  0: NP(NUM) *1
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
Quantities:
  0: NV(NUM) *1
  1: NP(NUM) *2
  2: QTE(NUM)
Wines:
  0: NV(NUM) *1
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
dummy:
  0: DEGRE(NUM)

Content of dummy
 * sqlite:///wine.db
Done.


DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


In [7]:
# Remove dummy table
%sql DROP TABLE IF EXISTS dummy;

 * sqlite:///wine.db
Done.


[]

__1.3__ Create the new tables from Master1:

In [8]:
# Write corresponding code here
# NV -> CRU, DEGREE, MILL

%sql DROP TABLE IF EXISTS Wines;
%sql CREATE TABLE Wines (NV NUM, CRU TEXT, DEGRE NUM, MILL NUM, PRIMARY KEY (NV));
%sql INSERT INTO Wines SELECT DISTINCT NV, CRU, DEGRE, MILL FROM Master1 WHERE NV NOT NULL;
%sql SELECT * FROM Wines LIMIT 5;

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
102 rows affected.
 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
2,Julienas,11.3,1974
3,Savigny les Beaunes,12.1,1978
4,Mercurey,10.9,1980
5,Pommard,11.7,1976


In [9]:
# NP -> NOM, PRENOM, REGION

%sql DROP TABLE IF EXISTS Producers;
%sql CREATE TABLE Producers (NP NUM, NOM TEXT, PRENOM TEXT, REGION TEXT, PRIMARY KEY (NP));
%sql INSERT INTO Producers SELECT DISTINCT NP, NOM, PRENOM, REGION FROM Master1 WHERE NP NOT NULL;
%sql SELECT * FROM Producers LIMIT 5

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
124 rows affected.
 * sqlite:///wine.db
Done.


NP,NOM,PRENOM,REGION
3,Six,Paul,Alsace
6,Marmagne,Bernard,Bourgogne
8,Lioger d'Harduy,Gabriel,Bourgogne
16,Barbin,Bernard,Bourgogne
17,Faiveley,Guy,Bourgogne


In [10]:
# NV, NP -> QTE

%sql DROP TABLE IF EXISTS Quantities;
%sql CREATE TABLE Quantities (NV NUM, NP NUM, QTE NUM, PRIMARY KEY (NV, NP));
%sql INSERT INTO Quantities SELECT DISTINCT NV, NP, QTE FROM Master1 WHERE NV NOT NULL AND NP NOT NULL;
%sql SELECT * FROM Quantities LIMIT 5;

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
140 rows affected.
 * sqlite:///wine.db
Done.


NV,NP,QTE
1,1,300
1,73,1
2,5,100
3,1,400
4,10,35


__1.4__ Create the new tables from Master2:

In [11]:
# Append the content of Wines from Master2 in the existing table Wines:
# NV -> CRU, DEGRE, MILL

%sql INSERT INTO Wines SELECT DISTINCT NV, CRU, DEGRE, MILL FROM Master2 \
WHERE NV NOT NULL AND NOT EXISTS (SELECT * FROM Wines);

%sql SELECT * FROM Wines LIMIT 5;

 * sqlite:///wine.db
0 rows affected.
 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
2,Julienas,11.3,1974
3,Savigny les Beaunes,12.1,1978
4,Mercurey,10.9,1980
5,Pommard,11.7,1976


In [12]:
# NB -> NOM, PRENOM, TYPE

%sql DROP TABLE IF EXISTS Clients;
%sql CREATE TABLE Clients (NB NUM, NOM TEXT, PRENOM TEXT, TYPE TEXT, PRIMARY KEY (NB));
%sql INSERT INTO Clients SELECT DISTINCT NB, NOM, PRENOM, TYPE FROM Master2 WHERE NB NOT NULL;
%sql SELECT * FROM Clients LIMIT 5;

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
100 rows affected.
 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen


In [13]:
# LIEU -> REGION

%sql DROP TABLE IF EXISTS Lieux;
%sql CREATE TABLE Lieux (LIEU TEXT, REGION TEXT, PRIMARY KEY (LIEU));
%sql INSERT INTO Lieux SELECT DISTINCT LIEU, REGION FROM Master2 WHERE LIEU NOT NULL;
%sql SELECT * FROM Lieux LIMIT 5;

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
18 rows affected.
 * sqlite:///wine.db
Done.


LIEU,REGION
BORDEAUX,NOUVELLE-AQUITAINE
PARIS,ÎLE-DE-FRANCE
RENNES,BRETAGNE
LYON,AUVERGNE-RHÔNE-ALPES
NICE,PROVENCE-ALPES-CÔTE D'AZUR


In [14]:
# NV, NB -> DATES, LIEU, QTE

%sql DROP TABLE IF EXISTS Buying;
%sql CREATE TABLE Buying (NV NUM, NB NUM, DATES TEXT, LIEU TEXT, QTE NUM, PRIMARY KEY (NV, NB));
%sql INSERT INTO Buying SELECT DISTINCT NV, NB, DATES, LIEU, QTE FROM Master2 WHERE LIEU NOT NULL;
%sql SELECT * FROM Buying LIMIT 5;

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
73 rows affected.
 * sqlite:///wine.db
Done.


NV,NB,DATES,LIEU,QTE
1,2,1977-11-02,BORDEAUX,33
1,44,2015-10-16,PARIS,1
1,45,1983-12-31,RENNES,1
2,48,1983-12-25,LYON,2
3,7,1978-11-01,NICE,6


# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

__2.1__ What are the different types of clients (buveurs) by volume of purchases?

In [15]:
%sql SELECT DISTINCT TYPE AS Type_of_buveurs FROM Clients;

 * sqlite:///wine.db
Done.


Type_of_buveurs
petit
moyen
gros


__2.2__ What regions produce Pommard or Brouilly?

In [16]:
%sql SELECT DISTINCT CRU, REGION FROM Wines, Producers, Quantities WHERE Quantities.NV = Wines.NV \
AND Quantities.NP = Producers.NP AND (CRU="Pommard" OR CRU="Brouilly");

 * sqlite:///wine.db
Done.


CRU,REGION
Pommard,Bourgogne
Pommard,Rhone
Brouilly,Bourgogne


__2.3__ What regions produce Pommard and Brouilly?

In [17]:
%sql SELECT DISTINCT REGION FROM Wines, Producers, Quantities WHERE Quantities.NV = Wines.NV \
AND Quantities.NP = Producers.NP AND CRU="Brouilly" \
INTERSECT SELECT DISTINCT REGION FROM Wines, Producers, Quantities WHERE Quantities.NV = Wines.NV \
AND Quantities.NP = Producers.NP AND CRU="Pommard";

 * sqlite:///wine.db
Done.


REGION
Bourgogne


__2.4__ Get the number of wines bought by CRU and Millésime

In [18]:
%sql SELECT CRU, MILL, QTE AS Number_of_wines_bought FROM Wines, Buying \
WHERE Buying.NV = Wines.NV GROUP BY CRU, MILL;

 * sqlite:///wine.db
Done.


CRU,MILL,Number_of_wines_bought
Arbois,1980,8
Auxey Duresses,1914,80
Beaujolais Primeur,1983,3
Beaujolais Villages,1975,10
Beaujolais Villages,1976,120
Beaujolais Villages,1978,130
Beaujolais Villages,1979,500
Chapelle Chambertin,1973,20
Chateau Corton Grancey,1980,4
Chenas,1984,1


__2.5__ Retrieve the wine number (NV) of wines produced by more than three producers

In [19]:
%sql SELECT NV, COUNT(*) AS Number_of_producers FROM Quantities \
GROUP BY NV HAVING Number_of_producers > 3;

 * sqlite:///wine.db
Done.


NV,Number_of_producers
45,5
78,5
89,4
98,5


__2.6__ Which producers have not produced any wine?

In [20]:
%sql SELECT NP, NOM, PRENOM FROM Producers \
WHERE NP NOT IN(SELECT NP FROM Quantities WHERE QTE NOT NULL);

 * sqlite:///wine.db
Done.


NP,NOM,PRENOM
3,Six,Paul
6,Marmagne,Bernard
8,Lioger d'Harduy,Gabriel
16,Barbin,Bernard
17,Faiveley,Guy
18,Tramier,Jean
19,Dupaquier,Roger
20,Lamy,Jean
21,Cornu,Edmond
26,Violot,Gilbert


__2.7__ What clients (buveurs) have bought at least one wine from 1980?

In [21]:
%sql SELECT DISTINCT Clients.NB, NOM, PRENOM FROM Buying, Clients, Wines \
WHERE Buying.NV=Wines.NV AND Buying.NB=Clients.NB AND MILL=1980;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
2,Artaud,Antonin
44,Gide,Andre
45,Giono,Jean
8,Aragon,Louis
50,Lautreamont,None
61,Mallarme,Stephane


__2.8__ What clients (buveurs) have NOT bought any wine from 1980?

In [22]:
%sql SELECT NB, NOM, PRENOM FROM Clients \
WHERE NB NOT IN(SELECT DISTINCT NB FROM Buying, Wines \
WHERE Buying.NV=Wines.NV AND MILL=1980) ORDER BY NB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
1,Aristote,None
3,Aron,Raymond
4,Apollinaire,Guillaume
5,Audiberti,Jacques
6,Arrabal,Fernando
7,Anouilh,Jean
9,Ajar,Emile
10,Andersen,Yann
11,Breton,Andre
12,Bataille,Georges


__2.9__ What clients (buveurs) have bought ONLY wines from 1980?

In [23]:
# Select clients that have bought at least one wine from 1980, and remove the clients that have
# also bought wines from other years:
%sql SELECT DISTINCT Clients.NB, NOM, PRENOM FROM Buying, Clients, Wines \
WHERE Buying.NV=Wines.NV AND Buying.NB=Clients.NB AND MILL=1980 \
AND Clients.NB NOT IN(SELECT DISTINCT NB FROM Buying, Wines \
WHERE Buying.NV=Wines.NV AND MILL!=1980);

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None


__2.10__ List all wines from 1980

In [24]:
%sql SELECT NV, CRU, MILL, DEGRE FROM Wines WHERE MILL=1980;

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
1,Mercurey,1980,11.5
4,Mercurey,1980,10.9
16,Meursault,1980,12.1
20,Cote de Brouilly,1980,12.1
26,Chateau Corton Grancey,1980,None
28,Volnay,1980,11
43,Fleurie,1980,11.4
74,Arbois,1980,12
78,Etoile,1980,12
79,Seyssel,1980,11


__2.11__ What are the wines from 1980 bought by NB=2?

In [25]:
%sql SELECT Wines.NV, NB, DATES, LIEU, QTE, CRU, MILL, DEGRE FROM Buying, Wines \
WHERE Buying.NV=Wines.NV AND MILL=1980 AND NB=2;

 * sqlite:///wine.db
Done.


NV,NB,DATES,LIEU,QTE,CRU,MILL,DEGRE
1,2,1977-11-02,BORDEAUX,33,Mercurey,1980,11.5


__2.12__ What clients (buveurs) have bought ALL the wines from 1980?

In [30]:
%sql SELECT Clients.NB, NOM, PRENOM FROM Buying, Clients \
WHERE Buying.NB=Clients.NB GROUP BY Clients.NB \
HAVING COUNT(NV)=(SELECT COUNT(NV) FROM Wines WHERE MILL=1980);

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
